In [23]:
import sys
sys.executable


'c:\\Users\\Ritwik\\OneDrive\\Desktop\\Work\\Inhouse Initiatives\\github_files_ritwik\\headstart_AI_projects\\week5_rate_my_professor\\Rate-My-Professor\\myenv\\Scripts\\python.exe'

In [24]:
from dotenv import load_dotenv
import os
import pinecone
# from cohere
from pinecone import Pinecone, ServerlessSpec
load_dotenv()

api_key = os.getenv("PINECONE_API_KEY")

if not api_key:
    raise ValueError("API key is not set in environment variables.")

pc = Pinecone(api_key=api_key)

index_names = pc.list_indexes().names()
if "rag" not in index_names:
    pc.create_index(
        name="rag",
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

print(pc.list_indexes().names())


['rag']


In [25]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Smith',
  'subject': 'Physics',
  'rating': 4,
  'review': "Dr. Smith's lectures are thorough and engaging. She explains complex topics clearly, but sometimes goes through material too quickly."},
 {'professor': 'Prof. Michael Johnson',
  'subject': 'Mathematics',
  'rating': 5,
  'review': 'Prof. Johnson is excellent! His teaching style is straightforward, and he always makes time for questions.'},
 {'professor': 'Dr. Sarah Davis',
  'subject': 'Chemistry',
  'rating': 3,
  'review': 'Dr. Davis is knowledgeable, but her lectures can be a bit hard to follow. Her office hours are very helpful though.'},
 {'professor': 'Prof. Robert Lee',
  'subject': 'History',
  'rating': 2,
  'review': "Prof. Lee's lectures lack structure and are often hard to follow. The course material is interesting, but he doesn't seem very organized."},
 {'professor': 'Dr. Laura Wilson',
  'subject': 'Biology',
  'rating': 4,
  'review': 'Dr. Wilson is a great professor. Her labs are wel

In [27]:
import os
import requests

HUGGINGFACE_API_URL = "https://api-inference.huggingface.co/pipeline/feature-extraction/sentence-transformers/all-MiniLM-L6-v2"
headers = {
    "Authorization": f"Bearer {os.getenv('HUGGINGFACE_API_KEY')}"
}

processed_data = []

for review in data['reviews']:
    response = requests.post(
        HUGGINGFACE_API_URL,
        headers=headers,
        json={"inputs": review['review']}
    )
    
    if response.status_code == 200:
        embeddings = response.json()[0]  # Extract embeddings
        processed_data.append({
            "values": embeddings,
            "id": review['professor'],
            "metadata": {
                "review": review["review"],
                "subject": review["subject"],
                "rating": review["rating"]
            }
        })
    else:
        print(f"Error: {response.status_code}, {response.text}")


In [17]:
from openai import OpenAI
from sentence_transformers import SentenceTransformer

processed_data = []
# client = OpenAI()
client = OpenAI(
    base_url="https://api-inference.huggingface.co/pipeline/feature-extraction/sentence-transformers/all-MiniLM-L6-v2",
    api_key=os.getenv("HUGGINGFACE_API_KEY")
)

for review in data['reviews']:
    # response = client.embeddings.create
    response = client.chat.completions.create(
        # input = review['review'],
        messages=review['review'],
        # model = "text-embedding-3-small",
        model="meta-llama/llama-3.1-8b-instruct:free",
        # model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2"),
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values" : embedding,
        "id": review['professor'],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "start": review["stars"]
        }
    })

ImportError: Error importing numpy: you should not try to import numpy from
        its source directory; please exit the numpy source tree, and relaunch
        your python interpreter from there.

In [6]:
from openai import OpenAI
# from sentence_transformers import SentenceTransformer

processed_data = []
# client = OpenAI()
client = OpenAI(
    # base_url="https://api-inference.huggingface.co/pipeline/feature-extraction/sentence-transformers/all-MiniLM-L6-v2",
    # api_key=os.getenv("HUGGINGFACE_API_KEY")
    base_url= "https://openrouter.ai/api/v1",
    api_key= os.getenv("OPENROUTER_API_KEY"),
)

for review in data['reviews']:
    # response = client.embeddings.create
    response = client.chat.completions.create(
        # input = review['review'],
        messages=review['review'],
        # model = "text-embedding-3-small",
        model="meta-llama/llama-3.1-8b-instruct:free",
        # model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2"),
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values" : embedding,
        "id": review['professor'],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "start": review["stars"]
        }
    })


BadRequestError: Error code: 400 - {'error': {'message': 'Input required: specify "prompt" or "messages"', 'code': 400}, 'user_id': 'user_2ke522P3pVl1iUmFNpYr80AudT0'}

In [ ]:
from openai import OpenAI
import os

processed_data = []
client = OpenAI(
    base_url="https://api-inference.huggingface.co/pipeline/feature-extraction/sentence-transformers/all-MiniLM-L6-v2",
    api_key=os.getenv("HUGGINGFACE_API_KEY")
)

for review in data['reviews']:
    response = client.completions.create(
        model="sentence-transformers/all-MiniLM-L6-v2",
        prompt=review['review'],
    )
    embedding = response.choices[0].text
    processed_data.append({
        "values": embedding,
        "id": review['professor'],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "start": review["stars"]
        }
    })

NotFoundError: Error code: 404 - {'error': 'Model sentence-transformers/all-MiniLM-L6-v2/completions does not exist'}

In [ ]:
from sentence_transformers import SentenceTransformer
import os
import numpy

# Initialize the SentenceTransformer model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

processed_data = []

for review in data['reviews']:
    # Generate embeddings for the review
    embedding = model.encode(review['review'])
    
    processed_data.append({
        "values": embedding.tolist(),  # Convert to list for JSON serialization
        "id": review['professor'],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "start": review["stars"]
        }
    })


ImportError: Error importing numpy: you should not try to import numpy from
        its source directory; please exit the numpy source tree, and relaunch
        your python interpreter from there.

In [28]:
print(embeddings.tolist())

AttributeError: 'float' object has no attribute 'tolist'

In [20]:
index = pc.Index('rag')
index.upsert(
    vectors = processed_data,
    namespace = "ns1"
)

ListConversionException: Expected a list or list-like data structure, but got: 0.029919113963842392

In [12]:
import openai
import os

# Initialize OpenRouter client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
)

processed_data = []

for review in data['reviews']:
    # Create an embedding using OpenRouter
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-ada-002"  # Or another valid embedding model
    )
    embedding = response['data'][0]['embedding']  # Adjust according to actual response format
    
    processed_data.append({
        "values": embedding,
        "id": review['professor'],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "start": review["stars"]
        }
    })


APIStatusError: Error code: 405

In [19]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}